In [28]:
%pip install ortools


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    # Calculate the great-circle distance between two points (in km)
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


In [30]:
class MCFLPD_3SH:
    def __init__(self, demand_points, facility_locations, p, num_drones):
        self.demand_points = demand_points  # list of tuples (lat, lon, demand)
        self.facility_locations = facility_locations  # list of tuples (lat, lon)
        self.p = p  # number of facilities to open
        self.num_drones = num_drones

        # Calculate distance matrix between demand points and facilities
        self.dist_matrix = np.zeros((len(demand_points), len(facility_locations)))
        for i, (dlat, dlon, _) in enumerate(demand_points):
            for j, (flat, flon) in enumerate(facility_locations):
                self.dist_matrix[i, j] = haversine_distance(dlat, dlon, flat, flon)

In [31]:


def solve_stage1(self):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Solver not available.")
        return None

    m = len(self.demand_points)
    n = len(self.facility_locations)

    x = {}
    y = {}

    for i in range(m):
        for j in range(n):
            x[i, j] = solver.BoolVar(f'x_{i}_{j}')
    for j in range(n):
        y[j] = solver.BoolVar(f'y_{j}')

    # Objective: minimize total weighted distance
    solver.Minimize(solver.Sum(
        self.demand_points[i][2] * self.dist_matrix[i, j] * x[i, j]
        for i in range(m) for j in range(n)
    ))

    # Constraints
    # Each demand point assigned to exactly one facility
    for i in range(m):
        solver.Add(solver.Sum(x[i, j] for j in range(n)) <= 1)

    # Assign only to open facilities
    for i in range(m):
        for j in range(n):
            solver.Add(x[i, j] <= y[j])

    # Open exactly p facilities
    solver.Add(solver.Sum(y[j] for j in range(n)) == self.p)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        self.y_solution = [int(y[j].solution_value()) for j in range(n)]
        self.x_solution = [[int(x[i, j].solution_value()) for j in range(n)] for i in range(m)]
        print("Stage 1 solved: Facility locations decided.")
    else:
        print("No optimal solution found for stage 1.")
        self.y_solution = None
        self.x_solution = None

MCFLPD_3SH.solve_stage1 = solve_stage1



In [32]:


def solve_stage2(self):
    if self.y_solution is None:
        print("Stage 1 not solved yet.")
        return

    open_facilities = [j for j, val in enumerate(self.y_solution) if val == 1]
    # Simple heuristic: divide drones equally among open facilities
    base = self.num_drones // len(open_facilities)
    remainder = self.num_drones % len(open_facilities)

    self.drones_per_facility = [0] * len(self.facility_locations)
    for idx, f in enumerate(open_facilities):
        self.drones_per_facility[f] = base + (1 if idx < remainder else 0)

    print(f"Stage 2 solved: Drone allocation to facilities = {self.drones_per_facility}")

MCFLPD_3SH.solve_stage2 = solve_stage2


In [33]:


def print_solution(self):
    if self.y_solution is None or self.x_solution is None:
        print("No solution available.")
        return

    print(f"Facilities opened at locations (indices): {[i for i, v in enumerate(self.y_solution) if v == 1]}")
    print(f"Drones allocated per facility: {self.drones_per_facility}")
    print("Demand point assignments (demand_point_index: facility_index):")
    for i, assignments in enumerate(self.x_solution):
        for j, assigned in enumerate(assignments):
            if assigned == 1:
                print(f"  Demand point {i} -> Facility {j}")

MCFLPD_3SH.print_solution = print_solution


In [36]:


demand_points = [
    (12.9716, 77.5946, 4.0),
    (13.0827, 80.2707, 3.0),
    (12.2958, 76.6394, 5.0),
    (10.4232, 100.5678, 6.0)
]

facility_locations = [
    (12.9611, 77.6387),
    (13.0350, 77.5970),
    (13.0010, 77.5700),
    (12.9141, 74.8560)
]

p = 2
num_drones = 4

problem = MCFLPD_3SH(demand_points, facility_locations, p, num_drones)
problem.solve_stage1()
problem.solve_stage2()
problem.print_solution()


TypeError: MCFLPD_3SH() takes no arguments